# Stereo Vision

In this notebook, the focus shifts from understanding epipolar geometry to recovering 3D scene information from multiple 2D images. One fundamental problem in multiple view geometry is **triangulation**, which involves determining the 3D location of a point by analyzing its projections in two or more images.

## Triangulation
**Triangulation Problem with Two Views:**
1. Two cameras with known intrinsic parameters `K` and `K'` are considered.
2. The relative orientation and translation between these cameras are known, denoted as `R` and `T`.
3. A 3D point `P` exists but is unknown, projected onto two images as `p` and `p'`.
4. By knowing `K`, `K'`, `R`, and `T`, we can calculate the lines of sight `l` and `l'` from the camera centers `O1` and `O2` to the image locations `p` and `p'`. The 3D point `P` is the intersection of these lines.

**Challenges in Triangulation:**
In practice, triangulation is more challenging due to several factors:
- The observed points `p` and `p'` are subject to noise.
- Camera calibration parameters might not be precise.
- The intersection of lines of sight may not exist due to noisy measurements.

**Linear Triangulation Method:**
To address the issues above, a simple linear triangulation method is used. Given two points `p` and `p'` corresponding to a 3D point `P`, we create three constraints for each point. These constraints result in a linear equation of the form `AP = 0`, where `A` is a matrix formed from the constraints. Solving this equation using Singular Value Decomposition (SVD) provides an estimate of the 3D point `P`.

**Nonlinear Triangulation:**
The linear method has limitations, particularly for projective reconstruction. In real-world scenarios, we often aim to minimize the reprojection error. This leads to a nonlinear minimization problem where we seek `P` that minimizes the sum of squared reprojection errors in all images:

```
min ∑ (∥MPˆ - p∥²)
 Pˆ
```

Solving this nonlinear least squares problem involves methods like the Gauss-Newton algorithm. This algorithm iteratively updates an estimate `Pˆ` by linearizing the residual error and finding the best correction to minimize the reprojection error.

**Handling Multiple Views:**
This approach naturally extends to multiple views. The linear least squares solution is augmented with the corresponding rows in the `e` vector and `J` matrix for each image, where `e` is the residual vector and `J` is the Jacobian matrix.

The Gauss-Newton algorithm is one way to iteratively refine the 3D point estimate. It repeats the linearization and correction steps for a fixed number of iterations or until convergence, with a limit on the number of updates to ensure practicality.

While the linear method is suitable for some cases, the nonlinear approach, especially with optimization algorithms like Gauss-Newton, offers more accurate results and handles multiple views effectively.

## **Affine Structure from Motion**

In this section, we dive into the concept of **structure from motion** (SfM), which is the process of simultaneously determining the 3D structure of a scene and the parameters of the cameras capturing that scene. The main goal is to recover both the camera transformation matrices (`Mi`) and the 3D points in the scene (`Xj`) from observations (`xij`) in multiple views.

**Affine Structure from Motion Problem:**
Before addressing the general SfM problem, we begin with a simpler version assuming affine or weak perspective cameras. In this case, the perspective scaling operation is not present, making mathematical derivations easier.

**Affine Camera Model:**
For the weak perspective model, the camera matrix `M` simplifies to a form where `v = 0`, which results in the homogeneous coordinate of `MX` being 1. This simplification allows us to express the projection as `AX + b`, where `A` is a 2x3 matrix, and `b` is a 2x1 vector.

**Data Centering Step:**
In the factorization method for affine SfM, we start with a data centering step. We redefine new coordinates `xˆij` for each image point `xij` by subtracting their centroid `x ̄i`:
```
xˆij = xij - x ̄i
```
This step centers the data at the origin in the image plane.

**Factorization Method:**
The key idea in factorization is to factorize the measurement matrix `D`, which comprises the centered observations, into two matrices: the motion matrix `M` (comprising camera matrices `A` and `b`) and the structure matrix `S` (comprising 3D points `Xj`). This factorization allows us to relate the 3D structure with their observed points compactly.

**Singular Value Decomposition (SVD):**
The factorization relies on SVD of the measurement matrix `D = UΣVT`. We use the three non-zero singular values from `Σ` to factorize `D` into `U3Σ3V3T`.

**Ambiguities in Reconstruction:**
A crucial point in SfM is that there is inherent ambiguity in any choice of the factorization `D = MS`. Specifically, any arbitrary, invertible 3x3 matrix `A` can be inserted into the decomposition, which leads to a multiplication of `M` and `S` by the same matrix `A`. This means the solution is underdetermined and requires extra constraints to resolve this affine ambiguity.

**Similarity Ambiguity:**
Another type of ambiguity in reconstruction is similarity ambiguity, where the reconstruction is correct up to a similarity transform (rotation, translation, and scaling). Calibrated cameras have only similarity ambiguity, meaning the absolute scale of the scene cannot be recovered from images.

In summary, SfM involves recovering both camera transformations and 3D scene points from image observations. In the affine SfM problem, we simplify the camera model, center the data, and use factorization methods. However, ambiguities exist in reconstruction, which may require additional constraints or information to resolve.

## **Perspective Structure from Motion**

Now, let's delve into the perspective structure from motion problem, which is more general and involves projective cameras (Mi). These cameras are described by 11 degrees of freedom in the form:

```
Mi = [a11 a12 a13 b1]
     [a21 a22 a23 b2]
     [a31 a32 a33  1]
```

It's important to note that solutions for both structure and motion can be determined up to a projective transformation in this general case. This means that you can apply a 4x4 projective transformation H to the motion matrix while also transforming the structure matrix by the inverse transformation H⁻¹, and the observations in the image plane remain the same.

**Unknowns in Perspective Structure from Motion:**
In this general perspective case, you have 11m + 3n - 15 unknowns (camera parameters and 3D points) and 2mn equations based on observations.

**Algebraic Approach:**
The algebraic approach aims to compute two camera matrices, M1 and M2, up to a perspective transformation H. The key steps include:

1. Compute the fundamental matrix F using the eight-point algorithm.
2. Use F to estimate the projective camera matrices M1 and M2.
3. Define the structure's 3D point as P and apply H⁻¹ to both camera projection matrices.
4. Relate pixel coordinates p and p' to the transformed structure Pe using M1 and M2.

**Decomposition Using Fundamental Matrix:**
You can decompose F as F = [b]×A, where [b]× represents the skew-symmetric cross product matrix. This decomposition allows you to find b and A:

- Compute b as a least square solution of Fb = 0, with ||b|| = 1.
- Calculate A as A = -[b]×F.

**Factorization of Camera Matrices:**
With b and A known, you can determine M1H⁻¹ and M2H⁻¹:

- M1H⁻¹ = [I 0]
- M2H⁻¹ = [A b]

**Geometrical Interpretation of b:**
b represents an epipole, which is a point in one image that maps to zero when transformed by the Fundamental matrix.

**Determining Motion from the Essential Matrix:**
For calibrated cameras, you can use the Essential matrix E, which encodes the extrinsic parameters (rotation and translation). E can be computed from the Fundamental matrix F and the intrinsic matrix K as E = KᵀFK.

**Decomposition of E:**
Decompose E into [t]×R, where [t]× is skew-symmetric. This decomposition yields two potential rotations (R) and translations (t).

- You can choose one of the four possible R, t pairings based on triangulation of points in front of both cameras.

**Triangulation for Correct R, t Pair:**
Triangulate multiple points, and select the R, t pairing that results in most of these points being in front of both cameras. Triangulation ensures the correct relative orientation and translation between the cameras.